In [201]:
import pandas as pd
path1="Resources/schools_complete.csv"
path2="Resources/students_complete.csv"

In [202]:
school_df = pd.read_csv(path1)
students_df = pd.read_csv(path2)
# school_df['School ID', 'school_name', 'type', 'size', 'budget']
# students_df['Student ID', 'student_name', 'gender', 'grade', 'school_name', 'reading_score', 'math_score']

In [203]:
# * Create a high level snapshot (in table form) of the district's key metrics, including:
#   * Total Schools
total_schools = len(school_df['school_name'].unique())
print(f"Total number of schools: {total_schools}")
#   * Total Students
total_students = len(students_df['Student ID'].unique())
print("Total number of students: "+ '{:,.0f}'.format(total_students))
#   * Total Budget
budget_sum = school_df['budget'].sum()
print("Total school budget: " + '$ {:,.0f}'.format(budget_sum))
#   * Average Math Score
avg_mathscore = students_df['math_score'].mean()
print("Average math score: " + '{:,.2f}'.format(avg_mathscore))
#   * Average Reading Score
avg_readingscore = students_df['reading_score'].mean()
print("Average reading score: " + '{:,.2f}'.format(avg_readingscore))

Total number of schools: 15
Total number of students: 39,170
Total school budget: $ 24,649,428
Average math score: 78.99
Average reading score: 81.88


In [204]:
#   * Percentage of students passing Math
math_pass_boolean = (students_df['math_score']>=70)
# returns a boolean vector: 1 for each passing student and 0 for those failed the class
math_pass_sum =math_pass_boolean.sum()
math_count = students_df['grade'].count()
math_fail = math_count - math_pass_sum
math_pass_percentage = math_pass_sum / math_count *100
math_fail_percentage =100 - math_pass_percentage
print("Math results: ") 
print("              " + '{:,.2f}%'.format(math_pass_percentage) + " passed vs. " + '{:,.2f}%'.format(math_fail_percentage) + " failed")
print("              " + '{:,.0f}'.format(math_pass_sum) + " students passed vs. " + '{:,.0f}'.format(math_fail)+ " students failed")
print("  * note: 70 and above pass") 

Math results: 
              74.98% passed vs. 25.02% failed
              29,370 students passed vs. 9,800 students failed
  * note: 70 and above pass


In [205]:
# * Calculate the percentage of students with a passing reading score (70 or greater)
reading_pass_boolean = (students_df['reading_score']>=70)
reading_pass_sum = reading_pass_boolean.sum()
reading_count = students_df['reading_score'].count()
reading_pass_perc = reading_pass_sum / reading_count * 100
reading_fail_perc = 100 - reading_pass_perc
reading_fail_sum = reading_count - reading_pass_sum
print("Reading results: ")
print("               " + '{:,.2f}%'.format(reading_pass_perc) + " passed vs " + '{:,.2f}%'.format(reading_fail_perc)+ " failed")
print("               " + '{:,.0f}'.format(reading_pass_sum) + " passed vs " + '{:,.0f}'.format(reading_fail_sum)+ " failed")
print("  * note: 70 and above pass") 

Reading results: 
               85.81% passed vs 14.19% failed
               33,610 passed vs 5,560 failed
  * note: 70 and above pass


In [206]:
# Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
overall_avg_score = (avg_mathscore + avg_readingscore)*.5
overall_avg_score=float(overall_avg_score)
print("Overall average score: " + '{:,.2f}'.format(overall_avg_score)+" *")
print("  * average (math +reading score)")

Overall average score: 80.43 *
  * average (math +reading score)


In [207]:
# Create a dataframe to hold the above results
data = {'Total Schools': [total_schools], 
        'Total Students':[total_students], 
        'Total Budget':[budget_sum], 
        'Avg Math Score':[avg_mathscore], 
        'Avg Reading Score':[avg_readingscore],
        '% Passing Math': [math_pass_percentage],
        '% Passing Reading' : [reading_pass_perc],
        'Overall Avg Score' : [overall_avg_score]}
Snapshot_df = pd.DataFrame(data)
Snapshot_df

,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Avg Score
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.431606


In [208]:
# Optional: give the displayed data cleaner formatting
# snapshot to be _f formatted
snap_f = Snapshot_df.copy()
snap_f['Total Students'] = snap_f['Total Students'].map("{:,.0f}".format)
snap_f['Total Budget'] = snap_f['Total Budget'].map("$ {:,.0f}".format)
snap_f['Avg Math Score'] = snap_f['Avg Math Score'].map("{:,.2f}".format)
snap_f['Avg Reading Score'] = snap_f['Avg Reading Score'].map("{:,.2f}".format)
snap_f['% Passing Math'] = snap_f['% Passing Math'].map("{:,.2f} %".format)
snap_f['% Passing Reading'] = snap_f['% Passing Reading'].map("{:,.2f} %".format)
snap_f['Overall Avg Score'] = snap_f['Overall Avg Score'].map("{:,.2f}".format)
snap_f

,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Avg Score
0,15,"39,170","$ 24,649,428",78.99,81.88,74.98 %,85.81 %,80.43


In [274]:
# * Create an overview table that summarizes key metrics about each school, including:
#   * Total Students
gbschool_stu = students_df.groupby(['school_name']).count()
total_students = gbschool_stu['Student ID']
# Total School Budget
# created a groupby object with a single column
total_budget = school_df.groupby(['school_name', 'type'])['budget'].sum()
# then turned it into a dictionary
total_budget = { "Total Budget" :total_budget}
# dictionary to dataframe
perstudent_df = pd.DataFrame(total_budget)
perstudent_df.head()
# another column to dictionary req'd for per student budget
total_students ={"Total Students" : total_students }
# another dictionary to DataFrame, req'd for per student budget
total_students_df = pd.DataFrame(total_students)
#  second column added to per student dataframe
perstudent_df = pd.merge(perstudent_df, total_students_df, on="school_name")
#  creating a column for per student budget
perstudent_df['Per Student Budget'] = perstudent_df['Total Budget']/perstudent_df['Total Students']
#   * School Type will also inserted in below dataframe created
# Summary School DataFrame 
sum_df = pd.DataFrame({'school_name' : school_df['school_name']})
merge_df= pd.merge(perstudent_df, sum_df, on='school_name')

,school_name,Total Budget,Total Students,Per Student Budget
0,Bailey High School,3124928,4976,628.0
1,Cabrera High School,1081356,1858,582.0
2,Figueroa High School,1884411,2949,639.0
3,Ford High School,1763916,2739,644.0
4,Griffin High School,917500,1468,625.0
5,Hernandez High School,3022020,4635,652.0
6,Holden High School,248087,427,581.0
7,Huang High School,1910635,2917,655.0
8,Johnson High School,3094650,4761,650.0
9,Pena High School,585858,962,609.0


In [344]:
merge_df.head()
type_df = pd.DataFrame({ 'school_name' : school_df['school_name'],'type':school_df['type']})
merged_df = pd.merge(merge_df, type_df, on='school_name')
merged_df.head()

,school_name,Total Budget,Total Students,Per Student Budget,type
0,Bailey High School,3124928,4976,628.0,District
1,Cabrera High School,1081356,1858,582.0,Charter
2,Figueroa High School,1884411,2949,639.0,District
3,Ford High School,1763916,2739,644.0,District
4,Griffin High School,917500,1468,625.0,Charter


In [552]:
#   * Average Math Score
grouped_math =students_df.groupby('school_name')['math_score'].mean()
# restoring index crucial for future merge
grouped_math.reset_index
avgmath_df = pd.DataFrame({'Avg Math Score' : grouped_math})
avgmath_df = avgmath_df.reset_index()
merged_avgmath_df = pd.merge(merged_df,avgmath_df, on = 'school_name')
merged_avgmath_df
sum_df=merged_avgmath_df.copy()
#   * Average Reading Score
grouped_reading = students_df.groupby('school_name')['reading_score'].mean()
avgreading_df = pd.DataFrame({'Avg Reading Score':grouped_reading})
merged_avgreading_df = pd.merge(merged_avgmath_df,avgreading_df, on='school_name')
merged_avgreading_df
sum_df = merged_avgreading_df.copy()
sum_df

,school_name,Total Budget,Total Students,Per Student Budget,type,Avg Math Score,Avg Reading Score
0,Bailey High School,3124928,4976,628.0,District,77.048432,81.033963
1,Cabrera High School,1081356,1858,582.0,Charter,83.061895,83.975780
2,Figueroa High School,1884411,2949,639.0,District,76.711767,81.158020
3,Ford High School,1763916,2739,644.0,District,77.102592,80.746258
4,Griffin High School,917500,1468,625.0,Charter,83.351499,83.816757
5,Hernandez High School,3022020,4635,652.0,District,77.289752,80.934412
6,Holden High School,248087,427,581.0,Charter,83.803279,83.814988
7,Huang High School,1910635,2917,655.0,District,76.629414,81.182722
8,Johnson High School,3094650,4761,650.0,District,77.072464,80.966394
9,Pena High School,585858,962,609.0,Charter,83.839917,84.044699


In [569]:
#   * % Passing Math
passread_df = students_df[students_df["math_score"]>=70].groupby("school_name", as_index='False').count()
passread_df=passread_df=passread_df.reset_index()
passmath_df =pd.DataFrame({ 'school_name':passread_df["school_name"],
                          'pass_math':passread_df["math_score"]})
merged_for_passmath = pd.merge(sum_df,passmath_df,on='school_name')
merged_for_passmath['pass_math_percentage'] = merged_for_passmath['pass_math']/merged_for_passmath['Total Students']*100
merged_for_passmath = merged_for_passmath.drop(columns='pass_math')
sum_df = merged_for_passmath.copy()
sum_df.head(3)

,school_name,Total Budget,Total Students,Per Student Budget,type,Avg Math Score,Avg Reading Score,pass_math_percentage
0,Bailey High School,3124928,4976,628.0,District,77.048432,81.033963,66.680064
1,Cabrera High School,1081356,1858,582.0,Charter,83.061895,83.975780,94.133477
2,Figueroa High School,1884411,2949,639.0,District,76.711767,81.158020,65.988471


In [579]:
#   * % Passing Reading
passread_df = students_df[students_df['reading_score']>=70].groupby('school_name').count()
passread_df = passread_df.reset_index()
passread_only=pd.DataFrame({'school_name' :passread_df['school_name'],
                            'pass_read':passread_df['reading_score'],})
merged_for_passread = pd.merge(sum_df,passread_only,on='school_name')
merged_for_passread['pass_read_percentage'] = merged_for_passread['pass_read']/merged_for_passread['Total Students']*100
merged_for_passread = merged_for_passread.drop(columns='pass_read')
sum_df = merged_for_passread.copy()
sum_df

,school_name,Total Budget,Total Students,Per Student Budget,type,Avg Math Score,Avg Reading Score,pass_math_percentage,pass_read_percentage
0,Bailey High School,3124928,4976,628.0,District,77.048432,81.033963,66.680064,81.933280
1,Cabrera High School,1081356,1858,582.0,Charter,83.061895,83.975780,94.133477,97.039828
2,Figueroa High School,1884411,2949,639.0,District,76.711767,81.158020,65.988471,80.739234
3,Ford High School,1763916,2739,644.0,District,77.102592,80.746258,68.309602,79.299014
4,Griffin High School,917500,1468,625.0,Charter,83.351499,83.816757,93.392371,97.138965
5,Hernandez High School,3022020,4635,652.0,District,77.289752,80.934412,66.752967,80.862999
6,Holden High School,248087,427,581.0,Charter,83.803279,83.814988,92.505855,96.252927
7,Huang High School,1910635,2917,655.0,District,76.629414,81.182722,65.683922,81.316421
8,Johnson High School,3094650,4761,650.0,District,77.072464,80.966394,66.057551,81.222432
9,Pena High School,585858,962,609.0,Charter,83.839917,84.044699,94.594595,95.945946


In [596]:
#   * Overall Passing Rate (Average of the above two)
sum_df['overall'] = (sum_df['pass_math_percentage']+sum_df['pass_read_percentage'])/2
# * Create a dataframe to hold the above results
sum_renamed_df = sum_df.rename(columns={'school_name':'School Name',
                                        'type':'School Type', 
                                        'pass_math_percentage':'% Passing Math',
                                        'pass_read_percentage':'% Passing Reading',
                                        'overall':'% Overall Passing Rate'})
sum_renamed_df = sum_renamed_df.set_index('School Name')
# sum_renamed_df 

In [625]:
# formatting tables
sum_f_df=sum_renamed_df.copy()
sum_f_df['Total Students'] = sum_f_df['Total Students'].map("{:,.0f}".format)
sum_f_df['Total Budget'] = sum_f_df['Total Budget'].map("$ {:,.0f}".format)
sum_f_df['Avg Math Score'] = sum_f_df['Avg Math Score'].map("{:,.2f}".format)
sum_f_df['Avg Reading Score'] = sum_f_df['Avg Reading Score'].map("{:,.2f}".format)
sum_f_df['% Passing Math'] = sum_f_df['% Passing Math'].map("{:,.2f} %".format)
sum_f_df['% Passing Reading'] = sum_f_df['% Passing Reading'].map("{:,.2f} %".format)
sum_f_df['% Overall Passing Rate'] = sum_f_df['% Overall Passing Rate'] .map("{:,.2f}".format)
sum_f_df.head(3)

,Total Budget,Total Students,Per Student Budget,School Type,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Bailey High School,"$ 3,124,928","4,976",628.0,District,77.05,81.03,66.68 %,81.93 %,74.31
Cabrera High School,"$ 1,081,356","1,858",582.0,Charter,83.06,83.98,94.13 %,97.04 %,95.59
Figueroa High School,"$ 1,884,411","2,949",639.0,District,76.71,81.16,65.99 %,80.74 %,73.36


In [627]:
# top performing schools sorted
sum_topsorted=sum_f_df.sort_values("% Overall Passing Rate", ascending=False)[:5]
sum_topsorted

,Total Budget,Total Students,Per Student Budget,School Type,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,"$ 1,081,356","1,858",582.0,Charter,83.06,83.98,94.13 %,97.04 %,95.59
Thomas High School,"$ 1,043,130","1,635",638.0,Charter,83.42,83.85,93.27 %,97.31 %,95.29
Griffin High School,"$ 917,500","1,468",625.0,Charter,83.35,83.82,93.39 %,97.14 %,95.27
Pena High School,"$ 585,858",962,609.0,Charter,83.84,84.04,94.59 %,95.95 %,95.27
Wilson High School,"$ 1,319,574","2,283",578.0,Charter,83.27,83.99,93.87 %,96.54 %,95.20


In [629]:
# bottom performing schools sorted
sum_bottomsorted=sum_f_df.sort_values("% Overall Passing Rate")[:5]
sum_bottomsorted

,Total Budget,Total Students,Per Student Budget,School Type,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,"$ 2,547,363","3,999",637.0,District,76.84,80.74,66.37 %,80.22 %,73.29
Figueroa High School,"$ 1,884,411","2,949",639.0,District,76.71,81.16,65.99 %,80.74 %,73.36
Huang High School,"$ 1,910,635","2,917",655.0,District,76.63,81.18,65.68 %,81.32 %,73.50
Johnson High School,"$ 3,094,650","4,761",650.0,District,77.07,80.97,66.06 %,81.22 %,73.64
Ford High School,"$ 1,763,916","2,739",644.0,District,77.10,80.75,68.31 %,79.30 %,73.80


In [689]:
# # Math average by grade
#  9th grade: creating groupby > df > series
ninth_df = students_df.loc[students_df["grade"] == "9th"].groupby("school_name")
ninth_mathavg = pd.DataFrame(ninth_df["math_score"].mean())
ninth_series = ninth_mathavg['math_score']
# 10th grade: creating groupby > df > series
tenth_df = students_df.loc[students_df["grade"] == "10th"].groupby("school_name")
tenth_mathavg = pd.DataFrame(tenth_df["math_score"].mean())
tenth_series = tenth_mathavg['math_score']
# 11th grade: creating groupby > df > series
eleventh_df = students_df.loc[students_df["grade"] == "11th"].groupby("school_name")
eleventh_mathavg = pd.DataFrame(eleventh_df["math_score"].mean())
eleventh_series = eleventh_mathavg['math_score']
# 12th grade: creating groupby > df > series
twelveth_df = students_df.loc[students_df["grade"] == "12th"].groupby("school_name")
twelveth_mathavg = pd.DataFrame(twelveth_df["math_score"].mean())
twelveth_series = twelveth_mathavg['math_score']

mathavg_grade_df = pd.DataFrame({"9th Math Avg": ninth_series,
"10th Math Avg": tenth_series,
"11th Math Avg": eleventh_series,
"12th Math Avg": twelveth_series})
mathavg_grade_df

,9th Math Avg,10th Math Avg,11th Math Avg,12th Math Avg
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [699]:
# formatting style
mathavg_grade_dff =mathavg_grade_df.copy()
mathavg_grade_dff['9th Math Avg']=mathavg_grade_dff['9th Math Avg'].map("{:,.2f}".format)
mathavg_grade_dff['10th Math Avg']=mathavg_grade_dff['10th Math Avg'].map("{:,.2f}".format)
mathavg_grade_dff['11th Math Avg']=mathavg_grade_dff['11th Math Avg'].map("{:,.2f}".format)
mathavg_grade_dff['12th Math Avg']=mathavg_grade_dff['12th Math Avg'].map("{:,.2f}".format)
mathavg_grade_dff

,9th Math Avg,10th Math Avg,11th Math Avg,12th Math Avg
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


In [713]:
# # Reading scores by grade
#  9th grade: creating groupby > df > series
rninth_df = students_df.loc[students_df["grade"] == "9th"].groupby("school_name")
rninth_mathavg = pd.DataFrame(rninth_df["reading_score"].mean())
rninth_series = rninth_mathavg['reading_score']
rninth_series
# 10th grade: creating groupby > df > series
rtenth_df = students_df.loc[students_df["grade"] == "10th"].groupby("school_name")
rtenth_mathavg = pd.DataFrame(rtenth_df["reading_score"].mean())
rtenth_series = rtenth_mathavg['reading_score']
rtenth_series
# # 11th grade: creating groupby > df > series
releventh_df = students_df.loc[students_df["grade"] == "11th"].groupby("school_name")
releventh_mathavg = pd.DataFrame(releventh_df["reading_score"].mean())
releventh_series = releventh_mathavg['reading_score']
releventh_series
# # 12th grade: creating groupby > df > series
rtwelveth_df = students_df.loc[students_df["grade"] == "12th"].groupby("school_name")
rtwelveth_mathavg = pd.DataFrame(twelveth_df["reading_score"].mean())
rtwelveth_series = rtwelveth_mathavg['reading_score']
rtwelveth_series
readavg_grade_df = pd.DataFrame({"9th Reading Avg": rninth_series,
                                 "10th Reading Avg": rtenth_series,
                                 "11th Reading Avg": releventh_series,
                                 "12th Reading Avg": rtwelveth_series})

readavg_grade_df

,9th Reading Avg,10th Reading Avg,11th Reading Avg,12th Reading Avg
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [717]:
# formatting results
read_formatted = readavg_grade_df.copy()
read_formatted['9th Reading Avg'] =read_formatted['9th Reading Avg'].map("{:,.2f}".format)
read_formatted['10th Reading Avg']=read_formatted['10th Reading Avg'].map("{:,.2f}".format)
read_formatted['11th Reading Avg']=read_formatted['11th Reading Avg'].map("{:,.2f}".format)
read_formatted['12th Reading Avg']=read_formatted['12th Reading Avg'].map("{:,.2f}".format)
read_formatted

,9th Reading Avg,10th Reading Avg,11th Reading Avg,12th Reading Avg
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


In [859]:
# binning for school spending vs score
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
scoreByBudget = sum_df[["Avg Math Score", "Avg Reading Score", "pass_math_percentage", "pass_read_percentage", "overall"]].groupby(pd.cut(sum_df["Per Student Budget"], bins=spending_bins, labels=group_names )).mean()
# a backup dataframe before formatting
scoreByBudget_raw = scoreByBudget.copy()
# formatting dataframe
scoreByBudget['Avg Math Score']=scoreByBudget['Avg Math Score'].map("{:,.2f}".format)
scoreByBudget['Avg Reading Score']=scoreByBudget['Avg Reading Score'].map("{:,.2f}".format)
scoreByBudget['pass_math_percentage']=scoreByBudget['pass_math_percentage'].map("{:,.2f}".format)
scoreByBudget['pass_read_percentage']=scoreByBudget['pass_read_percentage'].map("{:,.2f}".format)
scoreByBudget['overall']=scoreByBudget['overall'].map("{:,.2f}".format)
scoreByBudget = scoreByBudget.rename(columns={'pass_math_percentage':'Passing Math', 'pass_read_percentage':'Passing Reading', 'overall':'Passing Overall'})
scoreByBudget=scoreByBudget.reset_index(['Per Student Budget'])
scoreByBudget.head()

,Per Student Budget,Avg Math Score,Avg Reading Score,Passing Math,Passing Reading,Passing Overall
0,<$585,83.46,83.93,93.46,96.61,95.04
1,$585-615,83.60,83.89,94.23,95.90,95.07
2,$615-645,79.08,81.89,75.67,86.11,80.89
3,$645-675,77.00,81.03,66.16,81.13,73.65


In [780]:
size_bins = [0, 1000, 2000, 5000]
sizegroup_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
scoreBysize = sum_df[["Avg Math Score", "Avg Reading Score", "pass_math_percentage", "pass_read_percentage", "overall"]].groupby(pd.cut(sum_df["Total Students"], bins=size_bins, labels=sizegroup_names )).mean()
scoreBysize_raw=scoreBysize.copy()
scoreBysize['Avg Math Score']=scoreBysize['Avg Math Score'].map("{:,.2f}".format)
scoreBysize['Avg Reading Score']=scoreBysize['Avg Reading Score'].map("{:,.2f}".format)
scoreBysize['pass_math_percentage']=scoreBysize['pass_math_percentage'].map("{:,.2f}".format)
scoreBysize['pass_read_percentage']=scoreBysize['pass_read_percentage'].map("{:,.2f}".format)
scoreBysize['overall']=scoreBysize['overall'].map("{:,.2f}".format)
scoreBysize=scoreBysize.rename(columns={ 'pass_math_percentage':'Passing Math', 'pass_read_percentage':'Passing Reading', 'overall':'Passing Overall'})
scoreBysize

,Avg Math Score,Avg Reading Score,Passing Math,Passing Reading,Passing Overall
Total Students,,,,,
Small (<1000),83.82,83.93,93.55,96.10,94.82
Medium (1000-2000),83.37,83.86,93.60,96.79,95.20
Large (2000-5000),77.75,81.34,69.96,82.77,76.36


In [847]:
# creating a copy before replace: crucial
scorebytype=sum_df.copy()
# replacing string format type with integers for bins
scorebytype["type"] = scorebytype["type"].replace({"Charter": 0, "District":1})
type_bins = [0, 1, 2]
typegroup_names = ["Charter", "District"]
sbtype = scorebytype[["type", "Avg Math Score", "Avg Reading Score", "pass_math_percentage", "pass_read_percentage", "overall"]]
scoretype_df = sbtype.groupby('type',as_index=False).mean()
scoretype_df
scoretype_df["type"] = scoretype_df["type"].replace({0:"Charter", 1:"District"})
scoretype_df.set_index('type')

,Avg Math Score,Avg Reading Score,pass_math_percentage,pass_read_percentage,overall
type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
